In [1]:
import pandas as pd # Thank god
from sklearn import linear_model
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import mean_squared_error

import numpy as np

In [2]:
df_train = pd.read_csv('house-prices-advanced-regression-techniques\logtrain.csv').set_index('Id')
(df_train).head()
df_train_explan = df_train.drop(columns = 'SalePrice')
df_train_y = df_train['SalePrice']


df_bintrain = pd.get_dummies(df_train_explan.astype(str))

df_bintrain.head()

,MSSubClass_120,MSSubClass_160,MSSubClass_180,MSSubClass_190,MSSubClass_20,MSSubClass_30,MSSubClass_40,MSSubClass_45,MSSubClass_50,MSSubClass_60,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
Id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,1,0
2,0,0,0,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
3,0,0,0,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,1,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,1,0


# Part 2: Naive Data Processing

In [3]:
for colname, coldata in df_train_explan.iteritems():
    print(f'{colname} has', len(set(coldata)), 'unique features')


MSSubClass has 15 unique features
MSZoning has 5 unique features
LotFrontage has 369 unique features
LotArea has 1073 unique features
Street has 2 unique features
Alley has 3 unique features
LotShape has 4 unique features
LandContour has 4 unique features
Utilities has 2 unique features
LotConfig has 5 unique features
LandSlope has 3 unique features
Neighborhood has 25 unique features
Condition1 has 9 unique features
Condition2 has 8 unique features
BldgType has 5 unique features
HouseStyle has 8 unique features
OverallQual has 10 unique features
OverallCond has 9 unique features
YearBuilt has 112 unique features
YearRemodAdd has 61 unique features
RoofStyle has 6 unique features
RoofMatl has 8 unique features
Exterior1st has 15 unique features
Exterior2nd has 16 unique features
MasVnrType has 5 unique features
MasVnrArea has 335 unique features
ExterQual has 4 unique features
ExterCond has 5 unique features
Foundation has 6 unique features
BsmtQual has 5 unique features
BsmtCond has 5

In [122]:
# PART 2 QUESTION 4, TRAIN THE MODEL

from math import log, sqrt
train = [s.strip().split(",") for s in open('house-prices-advanced-regression-techniques\my_train.csv').readlines()]
train = train[1:]
dev = [s.strip().split(",") for s in open('house-prices-advanced-regression-techniques\my_dev.csv').readlines()]
dev = dev[1:]
test = [s.strip().split(",") for s in open('house-prices-advanced-regression-techniques\my_test.csv').readlines()]
test = test[1:]

categorical_features = [1,	4,	5,	6,	7,	8,	9,	10,	11,	12,	13,	14,	19,	20,	21,	22,	23,	25,	26,	27,	28,	29,	30,	31,	33,	37,	38,	39,	40,	51,	53,	55,	57,	60,	61,	62,	72,	73,	74]
numeric_features = [0,	2,	3,	15,	16,	17,	18,	24,	32,	34,	35,	36,	41,	42,	43,	44,	45,	46,	47,	48,	49,	50,	52,	54,	56,	58,	59,	63,	64,	65,	66,	67,	68,	69,	70,	71]
mapping = {}

def binarize(file_list, new = True, sales = True, smart = False):
    y = []
    new_data = []    
    for row in file_list:
        new_row = []
    
        if sales:
            y.append(log(float(row[-1])))
            end = -1
        else:
            end = None
            
        for j, x in enumerate(row[1:end]):
            feature = (j, x)
            if new:
                if feature not in mapping:
                    mapping[feature] = len(mapping) # insert a new feature into the index
                new_row.append(mapping[feature])
            else:
                if feature in mapping:
                    new_row.append(mapping[feature])
        new_data.append(new_row)

    bindata = np.zeros((len(new_data),len(mapping)))
    for i, row in enumerate(new_data):
        for x in row:
            bindata[i][x] = 1

    return bindata, y




# BINARIZE DATA
bindata_train, train_y = binarize(train, new = True)
bindata_dev, dev_y = binarize(dev, new = False)
bindata_test, test_y = binarize(test, new = False, sales = False)


        
# CALL LINEAR REGRESSION, FIT MODEL, PREDICT MODEL
lm = linear_model.LinearRegression()
fit_mod = lm.fit(bindata_train, train_y)
pred_y = fit_mod.predict(bindata_dev)
print('Mean Squared Error for part 4 is:', sqrt(mean_squared_error(dev_y, pred_y)))
        
        




Mean Squared Error for part 4 is: 0.15630755195257634


In [5]:

# PART 2 QUESTION 5 TOP 10 AND BOTTOM 10 CURRENTLY NOT PULLING CORRECTLY, LIKELY BECAUSE OF POOR MODEL FIT

top_bott = []
for i, w in enumerate(fit_mod.coef_):
    top_bott.append((i,w))

top_bott_sorted = sorted(top_bott, key = lambda x: x[1], reverse = True)

    
    
for i in range(5):
    # POSITIVE
    for key, vals in mapping.items():
        if vals == top_bott_sorted[i][0]:
            feat_val = key
    print('\nPositive Index:', top_bott_sorted[i][0], 'Positive Weight:', top_bott_sorted[i][1], 'Positive Feature:', feat_val)
    
    neg_i = -i -1
    # NEGATIVE
    for key, vals in mapping.items():
        if vals == top_bott_sorted[neg_i][0]:
            feat_val = key
    print('Neg Index:', top_bott_sorted[neg_i][0], 'Neg Weight:', top_bott_sorted[neg_i][1], 'Neg Feature:', feat_val)


Positive Index: 45 Positive Weight: 0.1502200765778792 Positive Feature: (45, '1710')
Neg Index: 551 Neg Weight: -0.195700683334053 Neg Feature: (1, 'C (all)')

Positive Index: 299 Positive Weight: 0.1447142007193873 Positive Feature: (16, '9')
Neg Index: 35 Neg Weight: -0.18695065472452088 Neg Feature: (35, '0')

Positive Index: 8 Positive Weight: 0.12834144476475076 Positive Feature: (8, 'AllPub')
Neg Index: 0 Neg Weight: -0.16406225528777732 Neg Feature: (0, '60')

Positive Index: 311 Positive Weight: 0.12749464883943112 Positive Feature: (48, '3')
Neg Index: 37 Neg Weight: -0.14604398612052277 Neg Feature: (37, '856')

Positive Index: 168 Positive Weight: 0.12378709421335773 Positive Feature: (16, '8')
Neg Index: 1803 Neg Weight: -0.12858627347290982 Neg Feature: (45, '968')


In [6]:
# PART 2 QUESTION 6, BIAS DIMENSION
print('My fit intercept is:', fit_mod.intercept_)

My fit intercept is: 11.735119279699791


In [7]:
from math import exp
pred_test = fit_mod.predict(bindata_test)

exp_test = exp(1) ** pred_test
genIds = np.arange(1461,1461+len(exp_test))

df = pd.DataFrame({'Id': genIds, 'SalePrice': exp_test}).set_index('Id')

df.to_csv('predict_test.csv')

# 3: Smarter Binarization

In [55]:
train_copy = [s.strip().split(",") for s in open(r'house-prices-advanced-regression-techniques\train_copy.csv', 'r').readlines()]


df_train_copy = pd.DataFrame(train_copy)
df_train_copy.columns = df_train_copy.iloc[0]
df_train_copy = df_train_copy[1:]
df_train_copy = df_train_copy.set_index('Id')


df_train_dummies = pd.get_dummies(df_train_copy.drop(['SalePrice'], axis = 1), dtype = 'float')


In [48]:
df_train_dummies.to_csv('test.csv')

In [59]:
df_train_copy = df_train_copy.infer_objects()
for col in df_train_copy.columns:
    print(col, type(col))



MSSubClass <class 'str'>
MSZoning <class 'str'>
LotFrontage <class 'str'>
LotArea <class 'str'>
Street <class 'str'>
LotShape <class 'str'>
LandContour <class 'str'>
Utilities <class 'str'>
LotConfig <class 'str'>
LandSlope <class 'str'>
Neighborhood <class 'str'>
Condition1 <class 'str'>
Condition2 <class 'str'>
BldgType <class 'str'>
HouseStyle <class 'str'>
OverallQual <class 'str'>
OverallCond <class 'str'>
YearBuilt <class 'str'>
YearRemodAdd <class 'str'>
RoofStyle <class 'str'>
RoofMatl <class 'str'>
Exterior1st <class 'str'>
Exterior2nd <class 'str'>
MasVnrType <class 'str'>
MasVnrArea <class 'str'>
ExterQual <class 'str'>
ExterCond <class 'str'>
Foundation <class 'str'>
BsmtQual <class 'str'>
BsmtCond <class 'str'>
BsmtExposure <class 'str'>
BsmtFinType1 <class 'str'>
BsmtFinSF1 <class 'str'>
BsmtFinType2 <class 'str'>
BsmtFinSF2 <class 'str'>
BsmtUnfSF <class 'str'>
TotalBsmtSF <class 'str'>
Heating <class 'str'>
HeatingQC <class 'str'>
CentralAir <class 'str'>
Electrical <cl

In [99]:

df_train_copy[df_train_copy.columns[numeric_features]] = df_train_copy[df_train_copy.columns[numeric_features]].apply(pd.to_numeric)


In [133]:
df_train_dummies = pd.get_dummies(df_train_copy.columns[categorical_features+numeric_features])
df_train_dummies

,1stFlrSF,2ndFlrSF,3SsnPorch,BedroomAbvGr,BldgType,BsmtCond,BsmtExposure,BsmtFinSF1,BsmtFinSF2,BsmtFinType1,...,SaleType,ScreenPorch,Street,TotRmsAbvGrd,TotalBsmtSF,Utilities,WoodDeckSF,YearBuilt,YearRemodAdd,YrSold
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
71,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
72,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
73,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [111]:
pd.get_dummies(df_train_copy)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1.00,60.0,RL,65.0,8450.0,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0.0,0.0,0.0,0.0,0.0,2.0,2008.0,WD,Normal,208500.00
2.00,20.0,RL,80.0,9600.0,Pave,Reg,Lvl,AllPub,FR2,Gtl,...,0.0,0.0,0.0,0.0,0.0,5.0,2007.0,WD,Normal,181500.00
3.00,60.0,RL,68.0,11250.0,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,0.0,0.0,0.0,0.0,0.0,9.0,2008.0,WD,Normal,223500.00
4.00,70.0,RL,60.0,9550.0,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,272.0,0.0,0.0,0.0,0.0,2.0,2006.0,WD,Abnorml,140000.00
5.00,60.0,RL,84.0,14260.0,Pave,IR1,Lvl,AllPub,FR2,Gtl,...,0.0,0.0,0.0,0.0,0.0,12.0,2008.0,WD,Normal,250000.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1456.00,60.0,RL,62.0,7917.0,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0.0,0.0,0.0,0.0,0.0,8.0,2007.0,WD,Normal,175000.00
1457.00,20.0,RL,85.0,13175.0,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0.0,0.0,0.0,0.0,0.0,2.0,2010.0,WD,Normal,210000.00
1458.00,70.0,RL,66.0,9042.0,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0.0,0.0,0.0,0.0,2500.0,5.0,2010.0,WD,Normal,266500.00
